# Kernal Used: Python 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pgmpy.estimators import HillClimbSearch,BicScore,BayesianEstimator
from pgmpy.models import BayesianModel
from matplotlib import pyplot as plt
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from sklearn.ensemble import RandomForestClassifier

#import BNfinder as bnf

In [2]:
df = pd.read_csv("dataAll.csv")
#df = df.drop("Unnamed: 0", axis = 1)
df = df.dropna()
print df.shape
df.head()

(1454, 9)


,zipcode,%Foreign,%Uninsured,%vehicle,%foodStamp,%poverty,urban,%PrematureBirth,@LowBirthWeight
0,10001,16.230927,6.313218,77.510070,51.290074,17.5,1,10.5,11.0
1,10002,16.301731,7.873643,76.876850,42.674401,27.8,1,8.9,6.6
2,10003,12.463227,5.066324,77.210118,48.017334,9.9,1,9.7,8.7
3,10004,15.177398,6.241787,76.513672,52.299606,4.8,1,8.7,6.7
4,10005,16.211251,5.774971,85.056784,49.311137,12.1,1,10.7,6.0


## Premature Birth without Teen Birth Rate

In [3]:
dfunpren = df[['%Uninsured','%vehicle','%foodStamp','%poverty',
          '%Foreign','urban','%PrematureBirth']]
print dfunpren.shape
dfunpren.head()

(1454, 7)


,%Uninsured,%vehicle,%foodStamp,%poverty,%Foreign,urban,%PrematureBirth
0,6.313218,77.510070,51.290074,17.5,16.230927,1,10.5
1,7.873643,76.876850,42.674401,27.8,16.301731,1,8.9
2,5.066324,77.210118,48.017334,9.9,12.463227,1,9.7
3,6.241787,76.513672,52.299606,4.8,15.177398,1,8.7
4,5.774971,85.056784,49.311137,12.1,16.211251,1,10.7


In [4]:
for i in dfunpren:
    dfunpren[i] = pd.qcut(dfunpren[i].rank(method='first'),4,labels=False)
print dfunpren.head()

/Users/emilypadvorac/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


   %Uninsured  %vehicle  %foodStamp  %poverty  %Foreign  urban  \
0           1         3           3         3         3      1   
1           2         3           3         3         3      1   
2           1         3           3         1         3      1   
3           1         3           3         0         3      1   
4           1         3           3         2         3      1   

   %PrematureBirth  
0                2  
1                1  
2                1  
3                1  
4                2  


In [5]:
dfunpren['%PrematureBirth'].value_counts()

3    364
0    364
2    363
1    363
Name: %PrematureBirth, dtype: int64

In [6]:
train,test=train_test_split(dfunpren,random_state=999,test_size=0.4)

In [7]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = hc.estimate()
print(best_model.edges())

[('urban', '%Foreign'), ('%Foreign', '%foodStamp'), ('%Foreign', '%vehicle'), ('%poverty', '%Uninsured'), ('%vehicle', '%poverty')]


In [8]:
best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of urban:
+----------+----------+
| urban(0) | 0.256849 |
+----------+----------+
| urban(1) | 0.239726 |
+----------+----------+
| urban(2) | 0.251142 |
+----------+----------+
| urban(3) | 0.252283 |
+----------+----------+
CPD of %PrematureBirth:
+--------------------+----------+
| %PrematureBirth(0) | 0.25     |
+--------------------+----------+
| %PrematureBirth(1) | 0.25     |
+--------------------+----------+
| %PrematureBirth(2) | 0.259132 |
+--------------------+----------+
| %PrematureBirth(3) | 0.240868 |
+--------------------+----------+
CPD of %Foreign:
+-------------+---------------------+---------------------+---------------------+---------------------+
| urban       | urban(0)            | urban(1)            | urban(2)            | urban(3)            |
+-------------+---------------------+---------------------+---------------------+---------------------+
| %Foreign(0) | 0.41228070175438597 | 0.29577464788732394 | 0.07174887892376682 | 0.21875             |
+------

In [9]:
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"])
print "Out of sample:",(test.loc[:,"%PrematureBirth"]==predicted_test["%PrematureBirth"]).mean()

Out of sample: 0.23539518900343642


In [10]:
predicted_train = best_model.predict(train.loc[:,"%Uninsured":"urban"]-1)
print "In sample:",(train.loc[:,'%PrematureBirth']==predicted_train['%PrematureBirth']).mean()
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"]-1)
print "Out of sample:",(test.loc[:,'%PrematureBirth']==predicted_test['%PrematureBirth']).mean()

In sample: 0.2591743119266055
Out of sample: 0.23539518900343642


## Low Birth Weight no Teen Birth Rate

In [11]:
dflbpre = df[['%Uninsured','%vehicle','%foodStamp','%poverty',
          '%Foreign','urban','@LowBirthWeight']]
print dflbpre.shape
dflbpre.head()

(1454, 7)


,%Uninsured,%vehicle,%foodStamp,%poverty,%Foreign,urban,@LowBirthWeight
0,6.313218,77.510070,51.290074,17.5,16.230927,1,11.0
1,7.873643,76.876850,42.674401,27.8,16.301731,1,6.6
2,5.066324,77.210118,48.017334,9.9,12.463227,1,8.7
3,6.241787,76.513672,52.299606,4.8,15.177398,1,6.7
4,5.774971,85.056784,49.311137,12.1,16.211251,1,6.0


In [12]:
for i in dflbpre:
    dflbpre[i] = pd.qcut(dflbpre[i].rank(method='first'),4,labels=False)
print dflbpre.head()

   %Uninsured  %vehicle  %foodStamp  %poverty  %Foreign  urban  \
0           1         3           3         3         3      1   
1           2         3           3         3         3      1   
2           1         3           3         1         3      1   
3           1         3           3         0         3      1   
4           1         3           3         2         3      1   

   @LowBirthWeight  
0                3  
1                1  
2                2  
3                1  
4                1  


/Users/emilypadvorac/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
dflbpre['@LowBirthWeight'].value_counts()

3    364
0    364
2    363
1    363
Name: @LowBirthWeight, dtype: int64

In [15]:
train,test=train_test_split(dflbpre,random_state=999,test_size=0.4)

In [16]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = hc.estimate()
print(best_model.edges())

[('urban', '%Foreign'), ('%Foreign', '%foodStamp'), ('%Foreign', '%vehicle'), ('%poverty', '%Uninsured'), ('%vehicle', '%poverty'), ('%vehicle', '@LowBirthWeight')]


In [17]:
best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
for cpd in best_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of urban:
+----------+----------+
| urban(0) | 0.256849 |
+----------+----------+
| urban(1) | 0.239726 |
+----------+----------+
| urban(2) | 0.251142 |
+----------+----------+
| urban(3) | 0.252283 |
+----------+----------+
CPD of %Foreign:
+-------------+---------------------+---------------------+---------------------+---------------------+
| urban       | urban(0)            | urban(1)            | urban(2)            | urban(3)            |
+-------------+---------------------+---------------------+---------------------+---------------------+
| %Foreign(0) | 0.41228070175438597 | 0.29577464788732394 | 0.07174887892376682 | 0.21875             |
+-------------+---------------------+---------------------+---------------------+---------------------+
| %Foreign(1) | 0.3201754385964912  | 0.15023474178403756 | 0.12556053811659193 | 0.38392857142857145 |
+-------------+---------------------+---------------------+---------------------+---------------------+
| %Foreign(2) | 0.2105263

In [18]:
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"])
print "Out of sample:",(test.loc[:,"@LowBirthWeight"]==predicted_test["@LowBirthWeight"]).mean()

Out of sample: 0.31099656357388317


In [19]:
predicted_train = best_model.predict(train.loc[:,"%Uninsured":"urban"]-1)
print "In sample:",(train.loc[:,'@LowBirthWeight']==predicted_train['@LowBirthWeight']).mean()
predicted_test = best_model.predict(test.loc[:,"%Uninsured":"urban"]-1)
print "Out of sample:",(test.loc[:,'@LowBirthWeight']==predicted_test['@LowBirthWeight']).mean()

In sample: 0.2672018348623853
Out of sample: 0.29896907216494845
